# State Probability Aggregation Large Example

This example demonstrates aggregated state probabilities in a larger closed queueing network with more jobs and classes.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.nc.*
import jline.solvers.mva.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.STD)

// Create larger network
val model = Network("myLargeModel")

// Block 1: nodes (larger network)
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.PS)
val node3 = Queue(model, "Queue2", SchedStrategy.FCFS)
val node4 = Queue(model, "Queue3", SchedStrategy.PS)
node3.setNumberOfServers(3) // Queue2 has 3 servers
node4.setNumberOfServers(2) // Queue3 has 2 servers

In [ ]:
// Block 2: classes (multiple classes with different populations)
val jobclass1 = ClosedClass(model, "Class1", 5, node1, 0) // 5 jobs
val jobclass2 = ClosedClass(model, "Class2", 3, node1, 0) // 3 jobs
val jobclass3 = ClosedClass(model, "Class3", 2, node1, 0) // 2 jobs

In [ ]:
// Set service times with different rates
val jobclasses = arrayOf(jobclass1, jobclass2, jobclass3)
val nodes = arrayOf(node1, node2, node3, node4)

// Delay: same service time for all classes
for (jobclass in jobclasses) {
    node1.setService(jobclass, Exp.fitMean(1.0))
}

// Queue1: different service rates
node2.setService(jobclass1, Exp.fitMean(1.0/2.0))
node2.setService(jobclass2, Exp.fitMean(1.0/3.0))
node2.setService(jobclass3, Exp.fitMean(1.0/4.0))

// Queue2: different service rates
node3.setService(jobclass1, Exp.fitMean(1.0))
node3.setService(jobclass2, Exp.fitMean(1.0/2.0))
node3.setService(jobclass3, Exp.fitMean(1.0/3.0))

// Queue3: different service rates
node4.setService(jobclass1, Exp.fitMean(1.0/3.0))
node4.setService(jobclass2, Exp.fitMean(1.0))
node4.setService(jobclass3, Exp.fitMean(1.0/2.0))

In [ ]:
// Block 3: routing - cyclic network
val P = model.initRoutingMatrix()

// Routing: Delay -> Queue1 -> Queue2 -> Queue3 -> Delay
for (jobclass in jobclasses) {
    P.set(jobclass, jobclass, node1, node2, 1.0) // Delay -> Queue1
    P.set(jobclass, jobclass, node2, node3, 1.0) // Queue1 -> Queue2
    P.set(jobclass, jobclass, node3, node4, 1.0) // Queue2 -> Queue3
    P.set(jobclass, jobclass, node4, node1, 1.0) // Queue3 -> Delay
}

model.link(P)

In [ ]:
// Solve with NC (CTMC may be too large)
println("Solving with NC solver for large network:")
try {
    val solverNc = NC(model)
    val avgTableNc = solverNc.avgTable
    println("NC Results:")
    avgTableNc.print()
    
    // Test state probability aggregation
    node2.setState(intArrayOf(-1, -1, -1)) // State ignored
    node3.setState(intArrayOf(2, 1, 0))    // Specific state
    node4.setState(intArrayOf(-1, -1, -1)) // State ignored
    
    val probAggr = solverNc.getProbAggr(node3)
    println("\nAggregated state probability for Queue2 state [2,1,0]: $probAggr")
    
} catch (e: Exception) {
    println("NC solver error: ${e.message}")
}

In [ ]:
// Compare with MVA solution
println("\nComparing with MVA solver:")
try {
    val solverMva = MVA(model)
    val avgTableMva = solverMva.avgTable
    println("MVA Results:")
    avgTableMva.print()
} catch (e: Exception) {
    println("MVA solver error: ${e.message}")
}